# Profiling in Jax

## Goals:

- Use the `Perfetto` profiler interface

## Concepts:

-

## Acknowledgments:

This example was based off of [Jax - Neural_Network_and_Data_Loading](https://jax.readthedocs.io/en/latest/notebooks/Neural_Network_and_Data_Loading.html)



In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
import jax
from jax import random
import numpy as np


layer_sizes = [784, 512, 512, 10]
step_size = 0.01
num_epochs = 8
batch_size = 128
n_targets = 10

In [2]:
from torchvision.datasets import MNIST
from profiling_util_functions import NumpyLoader, FlattenAndCast, init_network_params, one_hot, update, accuracy, init_network_params

mnist_dataset = MNIST('/tmp/mnist/', download=True, transform=FlattenAndCast())
training_generator = NumpyLoader(mnist_dataset, batch_size=batch_size, num_workers=0)


train_images = np.array(mnist_dataset.train_data).reshape(len(mnist_dataset.train_data), -1)
train_labels = one_hot(np.array(mnist_dataset.train_labels), n_targets)

# Get full test dataset
mnist_dataset_test = MNIST('/tmp/mnist/', download=True, train=False)
test_images = jnp.array(mnist_dataset_test.test_data.numpy().reshape(len(mnist_dataset_test.test_data), -1), dtype=jnp.float32)
test_labels = one_hot(np.array(mnist_dataset_test.test_labels), n_targets)


/opt/homebrew/anaconda3/envs/numpy_to_jax/lib/python3.12/site-packages/torchvision/datasets/mnist.py:76: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/opt/homebrew/anaconda3/envs/numpy_to_jax/lib/python3.12/site-packages/torchvision/datasets/mnist.py:66: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/opt/homebrew/anaconda3/envs/numpy_to_jax/lib/python3.12/site-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/opt/homebrew/anaconda3/envs/numpy_to_jax/lib/python3.12/site-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [3]:
import time
params = init_network_params(layer_sizes, random.key(0))

num_epochs = 10

with jax.profiler.trace("/tmp/jax-trace", create_perfetto_link=True):
    for epoch in range(num_epochs):
        start_time = time.time()
        for x, y in training_generator:
            y = one_hot(y, n_targets)
            params = update(params, x, y)
        epoch_time = time.time() - start_time

        train_acc = accuracy(params, train_images, train_labels)
        test_acc = accuracy(params, test_images, test_labels)
        print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
        print("Training set accuracy {}".format(train_acc))
        print("Test set accuracy {}".format(test_acc))

2024-05-19 19:34:13.463073: E external/xla/xla/python/profiler/internal/python_hooks.cc:398] Can't import tensorflow.python.profiler.trace


Epoch 0 in 3.27 sec
Training set accuracy 0.9157500267028809
Test set accuracy 0.9196999669075012
Epoch 1 in 2.94 sec
Training set accuracy 0.9371166825294495
Test set accuracy 0.9383999705314636


2024-05-19 19:34:21.849240: E external/xla/xla/python/profiler/internal/python_hooks.cc:398] Can't import tensorflow.python.profiler.trace


Open URL in browser: https://ui.perfetto.dev/#!/?url=http://127.0.0.1:9001/perfetto_trace.json.gz


127.0.0.1 - - [19/May/2024 19:34:54] code 404, message File not found
127.0.0.1 - - [19/May/2024 19:34:54] "POST /status HTTP/1.1" 404 -
127.0.0.1 - - [19/May/2024 19:34:54] "GET /perfetto_trace.json.gz HTTP/1.1" 200 -


KeyboardInterrupt: 